In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from igp2 import AgentState
from igp2.data.data_loaders import InDDataLoader
from igp2.data.episode import Frame
from igp2.data.scenario import InDScenario, ScenarioConfig
from igp2.opendrive.map import Map
from igp2.opendrive.plot_map import plot_map
from core.feature_extraction import FeatureExtractor, GoalDetector
from core.goal_generator import GoalGenerator
from core import feature_extraction
print(feature_extraction.__file__)

ModuleNotFoundError: No module named 'igp2'

In [ ]:
odr_results = pd.read_csv('../predictions/heckstrasse_trained_trees_test.csv')
odr_results.shape

In [ ]:
lanelet_results = pd.read_csv('../../GRIT-lanelet/predictions/heckstrasse_trained_trees_test.csv')
lanelet_results.shape

In [ ]:
odr_results = pd.read_csv('../predictions/frankenburg_trained_trees_test.csv')

In [ ]:
odr_results

In [ ]:
lanelet_results = pd.read_csv('../../GRIT-lanelet/predictions/frankenburg_trained_trees_test.csv')

In [ ]:
lanelet_results

In [ ]:
lanelet_results[['episode', 'agent_id']].drop_duplicates()

In [ ]:
odr_results[['episode', 'agent_id']].drop_duplicates()

In [ ]:
lanelet_agents = lanelet_results.agent_id.drop_duplicates()
odr_agents = odr_results.agent_id.drop_duplicates()

In [ ]:
lanelet_agents.isin(odr_agents).sum()

In [ ]:
lanelet_agents.shape

In [ ]:
odr_agents.isin(lanelet_agents).sum()

In [ ]:
odr_agents.shape

In [ ]:
lanelet_agents.loc[~lanelet_agents.isin(odr_agents)]

In [ ]:
lanelet_results.loc[lanelet_results.agent_id==75]

Why does opendrive have more vehicles? e.g agent 1. 
Different goal locations? - yes
Bicycles are also included - needs fixing

Why are some agents included in lanelet2 but not odr? e.g. 75 - vehicle misses goal slightly to the right. Should goal detection be based on dist along lane rather than pointgoal? or increase goal radius to match lane width

Baseline acc goes down at final point (1.0 of traj obs)

In [ ]:
%cd ..

In [ ]:
scenario_name = 'frankenburg'
scenario_map = Map.parse_from_opendrive(f"scenarios/maps/{scenario_name}.xodr")

scenario_config = ScenarioConfig.load(f"scenarios/configs/{scenario_name}.json")
scenario = InDScenario(scenario_config)

In [ ]:
frankenburg

In [ ]:
episode_idx = 5
episode = scenario.load_episode(episode_idx)
agent = episode.agents[75]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_map(scenario_map, ax=ax)
path = agent.trajectory.path
ax.plot(path[:, 0], path[:, 1])
ax.plot(*scenario_config.goals[0], 'o')

In [ ]:
scenario_name = 'heckstrasse'
scenario_map = Map.parse_from_opendrive(f"scenarios/maps/{scenario_name}.xodr")

scenario_config = ScenarioConfig.load(f"scenarios/configs/{scenario_name}.json")
scenario = InDScenario(scenario_config)

In [ ]:
goal_detector = GoalDetector(scenario.config.goals)

In [ ]:
episode_idx = 0
episode = scenario.load_episode(episode_idx)

In [ ]:
agent = episode.agents[0]

In [ ]:
agent_goals, goal_frame_idxes = goal_detector.detect_goals(agent.trajectory)

In [ ]:
agent_goals

In [ ]:
trajectory = agent.trajectory

In [ ]:
feature_extractor = FeatureExtractor(scenario_map)

In [ ]:
for idx in range(0, len(agent.trajectory.path)):
    typed_goals = feature_extractor.get_typed_goals(agent.trajectory.slice(0, idx+1), scenario.config.goals)
    print(idx, [g is not None for g in typed_goals])

In [ ]:
typed_goals

In [ ]:
agent.trajectory.path[68]

In [ ]:
ax = plot_map(scenario_map)
ax.plot(*agent.trajectory.path[68], 'o')
ax.plot([20],[-60], 'o')

In [ ]:
scenario_map.lanes_at(agent.trajectory.path[0], max_distance=3)

In [ ]:
lanes = scenario_map.lanes_within_angle(agent.trajectory.path[0],
                                        agent.trajectory.heading[0],
                                threshold=np.pi/4, max_distance=3)
print(lanes)

In [ ]:
ax = plot_map(scenario_map)
for lane in lanes:
    ax.plot(*list(zip(*[x for x in lane.midline.coords])))

In [ ]:
goal_point = np.array((62.2, -47.3))
idx = 70
best_lane = scenario_map.best_lane_at(agent.trajectory.path[idx],
                                  agent.trajectory.heading[idx],
                                  max_distance=3, goal_point=goal_point)
print(best_lane)

In [ ]:
ax = plot_map(scenario_map)
ax.plot(*list(zip(*[x for x in best_lane.midline.coords])))

In [ ]:
data = pd.read_csv('data/heckstrasse_e0.csv')

In [ ]:
goals_10 = data.loc[data.fraction_observed==1.0].value_counts('agent_id')

In [ ]:
goals_09 = data.loc[data.fraction_observed==0.9].value_counts('agent_id')

In [ ]:
(goals_10 > goals_09).sum()

In [ ]:
predictions = pd.read_csv('predictions/heckstrasse_prior_baseline_test.csv')
predictions

In [ ]:
predictions.loc[predictions.fraction_observed==1.0].model_correct.mean()

In [ ]:
predictions.loc[predictions.fraction_observed==0.9].model_correct.mean()

In [ ]:
idx = predictions.loc[predictions.fraction_observed==0.9].set_index('agent_id').model_correct \
                      != predictions.loc[predictions.fraction_observed==1.0].set_index('agent_id').model_correct

In [ ]:
idx.loc[idx]

In [ ]:
predictions.loc[predictions.agent_id==15]

Problem: Wrong goal type inferred at the last minute - why? G1 assigned goal type turn-left

In [ ]:
data.loc[data.agent_id==15]

In [ ]:
# lane id -1 on road 6, heckstrasse - detected as goal G1 - this must be junction NE to SE
# take into account trajectory history when detecting lane? Is this done for lanelet2 GRIT? e.g. previous lanelet

In [ ]:
ax = plot_map(scenario_map)
lane = scenario_map.get_lane(7, -1)
ax.plot(*list(zip(*[x for x in lane.midline.coords])))
ax.plot([36.0], [-27.0], 'o')

In [ ]:
heading = -0.6367160078810041
speed = 15.915689301070186

In [ ]:
scenario_name = 'round'
scenario_map = Map.parse_from_opendrive(f"scenarios/maps/{scenario_name}.xodr")

scenario_config = ScenarioConfig.load(f"scenarios/configs/{scenario_name}.json")
scenario = InDScenario(scenario_config)

In [ ]:
episode_idx = 0
episode = scenario.load_episode(episode_idx)


In [ ]:
print(len(agent_goals))
for g in agent_goals:
    print(g)

In [ ]:
pwd

In [ ]:
odr_results = pd.read_csv('predictions/round_trained_trees_test.csv')
odr_results.shape

In [ ]:
lanelet_results = pd.read_csv('../GRIT-lanelet/predictions/round_trained_trees_test.csv')
lanelet_results.shape

In [ ]:
odr_results[['episode', 'agent_id', 'fraction_observed']]

In [ ]:
# isin with multiple columns?

In [ ]:
episode = 4
fraction_observerd = 0.8
odr_samples = odr_results.loc[(odr_results.episode == episode) 
                & (odr_results.fraction_observed == fraction_observerd)].set_index('agent_id')
lanelet_samples = lanelet_results.loc[(lanelet_results.episode == episode) 
                & (lanelet_results.fraction_observed == fraction_observerd)].set_index('agent_id')

In [ ]:
lanelet_samples

In [ ]:
odr_samples

In [ ]:
odr_samples = odr_samples.join(lanelet_samples.model_correct, rsuffix='_ll')

In [ ]:
odr_samples.loc[odr_samples.model_correct != odr_samples.model_correct_ll]